# Problema de Transporte

O problema de transporte se refere a distribuir qualquer produto de qualquer grupo de centros de fornecimento, chamados **origens**, a qualquer grupo de centros de recepção, denominados **destinos**, de modo a minimizar o custo total de distribuição.

<img src="./img/PT-tabela-geral.png">

## Variáveis do Problema
- $x$ número de unidades do produto
- $m$ origens
- $n$ destinos
- Oferta $s_i$ da origem $i$
- Demanda $d_j$ no destino $j$
- Custo $c_{ij}$ por unidade distribuída da origem $i$ para o destino $j$

        Os únicos dados necessários para um modelo de problema de transporte são as origens, demandas e custos unitários.

### Hipótese das exigências: 

$\sum_{i=1}^m s_i = \sum_{j=1}^n d_j$

Cada origem tem uma oferta fixa de unidades, em que toda essa oferta tem de ser distribuída aos destinos. (Façamos que $s_i$ represente o número de unidades sendo distribuídas pela origem $i$, para $i = 1, 2, ... , m.$) 

De forma similar, cada destino tem uma demanda fixa por unidades, nas quais toda essa demanda deve ser recebida das origens. (Façamos que $d_j$ represente o número de unidades recebidas pelo destino $j$, para $j = 1, 2, ... , n.$)

Em problemas reais, as ofertas, na verdade, representam quantidades máximas (e não quantidades fixas) a serem distribuídas. De modo similar, em outros casos, as demandas representam quantidades máximas (e não quantidades fixas) a serem recebidas. 

Tais problemas violam a **hipótese das exigências**. Entretanto, é possível reformular o problema de modo que eles atendam a esse modelo, pela introdução de um destino "fantasma" ou de uma origem "fantasma" para absorver a folga entre as quantidades reais e as quantidades máximas que estão sendo distribuídas. 

### Hipótese do custo: 

O custo de distribuição de unidades de qualquer origem em particular para qualquer destino em particular é **diretamente proporcional** ao número de unidades distribuídas. 

Portanto, esse custo é simplesmente o custo unitário de distribuição vezes o número de unidades distribuídas. 

## Modelo do Problema:

Fazendo que $Z$ seja o custo total de distribuição e $x_{ij} (i= 1,2,...,m;j= 1,2,...,n)$ seja o número de unidades a serem distribuídas da origem $i$ para o destino $j$, a formulação em programação linear desse problema ficaria assim

Minimizar: $Z = \sum_{i=1}^m \sum_{j=1}^n c_{ij}x_{ij}$


Sujeito a: 

$\sum_{j=1}^n x_{ij} = s_i$, para $j= 1,2,...,m$,

$\sum_{i=1}^m x_{ij} = d_j$, para $i= 1,2,...,n$,

$x_{ij} \ge 0$, para todo $i$ e $j$.

$x_{ij}$ inteiro, para todo $i$ e $j$.

### Propriedade das soluções inteiras: 

Para problemas de transporte em que todos $s_i$ e $d_j$ são valores inteiros, todas as variáveis básicas (alocações) em toda solução (BV) viável (inclusive uma solução ótima) também são valores inteiros.

Para muitas aplicações, os volumes de oferta $s_i$ e de demanda $d_j$ no modelo têm valores inteiros e a implementação exigirá que as quantidades distribuídas $x_{ij}$ também sejam valores inteiros.

O MÉTODO SIMPLEX APERFEIÇOADO PARA O PROBLEMA DE TRANSPORTE lida somente com soluções BV inteiras, de modo que ele vai obter automaticamente uma solução ótima inteira. Portanto, é desnecessário acrescentar uma restrição ao modelo em que os $x_{ij}$ têm de ser inteiros.

## Problema de exemplo

Um dos principais produtos da P & T COMPANY são ervilhas enlatadas. As ervilhas pre- paradas em três fábricas de enlatados (próximas a Bellingham, Washington; Eugene, Oregon; e Albert Lea, Minnesota) e depois transportadas por caminhão para quatro depósi- tos de distribuição no oeste dos Estados Unidos (Sacramento, Califórnia; Salt Lake City, Utah; Dakota do Sul; e Albuquerque, Novo México).

<img src="./img/PT-mapa.png">

Para a próxima temporada, foi feita uma estimativa do volume proveniente de cada fábrica de enlatados e foi destinado a cada depósito certa quantidade do suprimento total de ervilhas. Essas informações (em unidades de carretas), juntamente com o custo de transporte por carreta para cada combinação fábrica-depósito.

<img src="./img/PT-grafo.png">

Há uma carga total a ser remetida de 300 carretas. O problema agora é determinar qual plano de destinação dessas remessas às diversas combinações fábrica-depósito iria minimizar o custo total de remessa dessa mercadoria.

<img src="./img/PT-tabela.png">

**Minimizar:** $Z = 464x_{11} + 513x_{12} + 654x_{13} + 867x_{14} + 352x_{21} + 416x_{22} + 690x_{23} + 791x_{24} + 995x_{31} + 682x_{32} + 388x_{33} + 685x_{34}$

**Sujeito a:**

<img src="./img/PT-matriz.png">

In [2]:
#include <iostream>
#include <iomanip>
#include "setup.h"
#include "ortools/linear_solver/linear_solver.h"
#include "ortools/linear_solver/linear_solver.pb.h"

using namespace operations_research;

const double infinity = MPSolver::infinity();

In [3]:
{
    const int m(3), n(4);
    
    int s[m] = {75, 125, 100};

    int d[n] = {80, 65, 70, 85};

    int c[m][n] = {{464,513,654,867}, 
                   {352,416,690,791}, 
                   {995,682,388,685}};
    
    MPSolver solver("transporte", MPSolver::GLOP_LINEAR_PROGRAMMING);
    
    std::vector<std::vector<MPVariable*>> x(m, std::vector<MPVariable*>(n));
    
    for(int i=0; i < m; i++) {
        for(int j=0; j < n; j++) {
            x[i][j] = solver.MakeNumVar(0.0, infinity, "x" + std::to_string(i+1) + std::to_string(j+1));
        }
    }
    std::cout << "Number of variables = " << solver.NumVariables() << std::endl;
    
    // Restrição de Origem
    for(int i=0; i < m; i++) {
        MPConstraint* const c = solver.MakeRowConstraint(s[i], s[i], "s" + std::to_string(i+1));
        for(int j=0; j < n; j++) {
            c->SetCoefficient(x[i][j], 1);
        }
    }

    // Restrição de Destino
    for(int j=0; j < n; j++) {
        MPConstraint* const c = solver.MakeRowConstraint(d[j], d[j], "d" + std::to_string(j+1));
        for(int i=0; i < m; i++) {
            c->SetCoefficient(x[i][j], 1);
        }
    }
    std::cout << "Number of constraints = " << solver.NumConstraints() << std::endl;
    
    // Minimizar Z.
    MPObjective* const objective = solver.MutableObjective();
    for(int i=0; i < m; i++) {
        for(int j=0; j < n; j++) {
            objective->SetCoefficient(x[i][j], c[i][j]);
        }
    }
    objective->SetMinimization();
    
    const MPSolver::ResultStatus result_status = solver.Solve();
    // Check that the problem has an optimal solution.
    if (result_status != MPSolver::OPTIMAL) {
         std::cout << "The problem does not have an optimal solution!" << std::endl;
    }
    
    std::cout << "Solution:" << std::endl;
    std::cout << "Objective value = " << objective->Value() << std::endl;
    for(int i=0; i < m; i++) {
        for(int j=0; j < n; j++) {
            std::cout << " x" << i+1 << j+1 << "=" << std::setw(3) << x[i][j]->solution_value() ;
        }
        std::cout << std::endl;
    }
 
}

Number of variables = 12
Number of constraints = 7
Solution:
Objective value = 152535
 x11=  0 x12= 20 x13=  0 x14= 55
 x21= 80 x22= 45 x23=  0 x24=  0
 x31=  0 x32=  0 x33= 70 x34= 30


In [4]:
//MODIFIQUE O EXEMPLO ACIMA PARA (1) USAR O SOLVER SCIP (2) COM VARIAVEIS INTEIRAS E (3) VERIFICAR SE A PROPRIEDADE DE SOLUÇÕES INTEIRAS É SATISFEITA

## Exemplo com um Destino "Fantasma"

A CIA. AÉREA SETENTRIONAL constrói aviões comerciais para diversas companhias aéreas ao redor do mundo. O último estágio no processo de produção é produzir os motores a jato e depois instalá-los (uma operação muito rápida) na estrutura completa da aeronave.

<img src="./img/setentrional.png">

- Origem i = produção de motores ajato no mês i (i = 1, 2, 3, 4) 
- Destino j = instalação de motores a jato no mês j (j = 1, 2, 3, 4)
- $x_{ij}$ = número de motores produzido no mês i para instalação no mês j 
- $c_{ij}$ = Custo por unidade para produção e armazenamento se $i \le j$
- $s_i$ =?
- $d_j$ = número de instalações programadas no mês j.

In [7]:
{
    const int m(4), n(4);
    
    const double M = 100;
    
    double s[m] = {25, 35, 30, 10};
    
    double d[n] = {10, 15, 25, 20};

    double c[m][n] = {{1.080,1.095,1.110,1.125}, 
                      {M,1.110,1.125,1.140}, 
                      {M,M,1.100,1.115},
                      {M,M,M,1.130}};
    
    MPSolver solver("transporte2", MPSolver::SCIP_MIXED_INTEGER_PROGRAMMING);
    
    std::vector<std::vector<MPVariable*>> x(m, std::vector<MPVariable*>(n));
    
    for(int i=0; i < m; i++) {
        for(int j=0; j < n; j++) {
            x[i][j] = solver.MakeIntVar(0, (c[i][j] == M ? 0 : infinity), 
                                        "x" + std::to_string(i+1) + std::to_string(j+1));
        }
    }
    std::cout << "Number of variables = " << solver.NumVariables() << std::endl;
    
    // Restrição de Origem
    for(int i=0; i < m; i++) {
        MPConstraint* const c = solver.MakeRowConstraint(-infinity, s[i], "s" + std::to_string(i+1));
        for(int j=0; j < n; j++) {
            c->SetCoefficient(x[i][j], 1);
        }
    }

    // Restrição de Destino
    for(int j=0; j < n; j++) {
        MPConstraint* const c = solver.MakeRowConstraint(d[j], d[j], "d" + std::to_string(j+1));
        for(int i=0; i < m; i++) {
            c->SetCoefficient(x[i][j], 1);
        }
    }
    std::cout << "Number of constraints = " << solver.NumConstraints() << std::endl;
    
    // Minimizar Z.
    MPObjective* const objective = solver.MutableObjective();
    for(int i=0; i < m; i++) {
        for(int j=0; j < n; j++) {
            objective->SetCoefficient(x[i][j], c[i][j]);
        }
    }
    objective->SetMinimization();
    
    const MPSolver::ResultStatus result_status = solver.Solve();
    // Check that the problem has an optimal solution.
    if (result_status != MPSolver::OPTIMAL) {
         std::cout << "The problem does not have an optimal solution!" << std::endl;
    }
    
    std::cout << "Solution:" << std::endl;
    std::cout << "Objective value = " << objective->Value() << std::endl;
    for(int i=0; i < m; i++) {
        for(int j=0; j < n; j++) {
            std::cout << " x" << i+1 << j+1 << "=" << std::setw(3) << x[i][j]->solution_value() ;
        }
        std::cout << std::endl;
    }
 
}

Number of variables = 16
Number of constraints = 8
Solution:
Objective value = 77.3
 x11= 10 x12= 10 x13=  5 x14=  0
 x21= -0 x22=  5 x23=  0 x24=  0
 x31= -0 x32= -0 x33= 20 x34= 10
 x41= -0 x42= -0 x43= -0 x44= 10
